In [2]:
import pandas as pd
import numpy as np
import requests
import os 
from dotenv import load_dotenv
import requests
from tqdm import tqdm
import pickle
import json
pd.options.display.max_columns = None

In [3]:
import spotipy 
from spotipy.oauth2 import SpotifyClientCredentials ## para poner mis credenciales

# Importing pickle file

In [4]:
with open('../data/pickle/spotify_dataset.pickle', 'rb') as tracks:
    tracks = pickle.load(tracks)

In [5]:
tracks.head()

,artist_clean,track_clean
0,elvis costello,(the angels wanna wear my) red shoes
1,elvis costello and the attractions,"(what's so funny 'bout) peace, love and unders..."
2,tiffany page,7 years too late
3,elvis costello and the attractions,accidents will happen
4,elvis costello,alison


# spotify
Aquí está la docu: https://spotipy.readthedocs.io/en/2.22.0/#

Configuro el cliente y le metó las keys para acceder a la API de Spotify y poder sacar datos de las canciones que tengo

In [6]:
load_dotenv()

True

In [7]:
id = os.getenv("id")
secret = os.getenv("secret")

client_credentials_manager = SpotifyClientCredentials(client_id=id, client_secret=secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager, requests_timeout=10, retries=10)

Primero voy a sacar el URI de mis canciones. Para ello uso el método de spotipy.search, que permite buscar con una query el nombre del artista y la canción.

In [8]:
## esto es para obtener una uri --> me voy a sacar los uris para mi data frame
artist = 'los del rio'
track = 'la macarena'
q = 'artist:{} track: {}'.format(artist, track)
results = sp.search(q=q, limit=1, type='track')
print(results['tracks']['items'][0]['uri'])

In [10]:
for index, row in tracks.iterrows():
    print(row['artist_clean'],'---', row['track_clean'])
    break

elvis costello --- (the angels wanna wear my) red shoes


In [11]:
tqdm.pandas()

uris = []
for index, row in tqdm(tracks.iterrows(), total = tracks.shape[0]):
    q = 'artist:{} track: {}'.format(row['artist_clean'], row['track_clean'])
    res = sp.search(q=q, limit=1, type='track')
    try:
        uri = res['tracks']['items'][0]['uri']
        #print(res['tracks']['items'][0]['uri'])
        uris.append(uri)
    except:
        uris.append('')
    #break

  0%|          | 0/2691039 [00:00<?, ?it/s]

Voy a comprobar que la lista donde me he appendeado todo tiene la misma longitud que mi dataframe, para poder añadir los URIs como una columna

In [ ]:
uris[:3]

In [ ]:
print(len(uris), tracks.shape)

In [ ]:
tracks['URI'] = uris
tracks.head()

Ahora con el método `'audio_features'` de spotipy quiero sacar las propiedades que tienen las canciones. Para ello, tengo que aportar:
- `URI`

Y el resultado de cada una de mis consultas, lo almacenaré en una lista de listas que luego appendearé a mi dataframe.

In [ ]:
sp.audio_features('spotify:track:3OHfY25tqY28d16oZczHc8')

In [ ]:
type(sp.audio_features('spotify:track:3OHfY25tqY28d16oZczHc8'))

In [ ]:
datos_uris = []
for index, row in tracks.iterrows():
    uri = sp.audio_features(row['URI'])
    datos_uris.append(uri)

Y tengo mis datos sobre las canciones, pero tengo que limpiar un poco para poder hacer el append.

In [ ]:
type(pd.DataFrame(datos_uris).iloc[0,0]) ## puedo sacar las claves para nombrar a mis columnas

In [ ]:
columnas = list(pd.DataFrame(datos_uris).iloc[0,0].keys())
columnas[:5]

In [ ]:
df_uris = pd.DataFrame(datos_uris)
df_uris[columnas] = df_uris[0].apply(pd.Series)
df_uris.head()

Lo junto ahora con mi dataframe inicial y limpio las columnas que no necesite.

In [ ]:
tracks = tracks.merge(df_uris, left_on = 'URI', right_on='uri', indicator=True)
tracks['_merge'].value_counts() ## compruebo que el cruce se haga con éxito

In [ ]:
tracks.drop([0, 'uri', '_merge'], axis = 1, inplace=True)
tracks.head()

# Export

In [ ]:
with open('../data/spotify_dataset_spotify.pickle', 'wb') as data_spotify:
    pickle.dump(tracks, data_spotify)